In [1]:
import pandas as pd
import chardet
import numpy as np

def read_csv_auto_encoding(file_path, delimiter):
    # Phát hiện mã hóa tự động
    with open(file_path, 'rb') as f:
        result = chardet.detect(f.read(10000))
    
    encoding = result['encoding']
    print(f"Detected encoding: {encoding}")
    
    # Đọc tệp CSV với mã hóa được phát hiện
    try:
        df = pd.read_csv(file_path, delimiter=delimiter, encoding=encoding)            
        return df
    except Exception as e:
        print(f"Error with detected encoding {encoding}:", e)
        # Thử mã hóa phổ biến khác
        for enc in ['latin1', 'iso-8859-1', 'cp1252']:
            try:
                df = pd.read_csv(file_path, delimiter=delimiter, encoding=enc)
                print(f"Successfully read the file with encoding: {enc}")
                return df
            except Exception as e:
                print(f"Error with encoding {enc}:", e)
                
        return None

In [4]:
data = read_csv_auto_encoding("data/merged_books2.csv" , ",")

print(data.info())

Detected encoding: utf-8


C:\Users\buida\AppData\Local\Temp\ipykernel_9876\3654919718.py:15: DtypeWarning: Columns (4,5,6,12,13,14,15,16) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path, delimiter=delimiter, encoding=encoding)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 374298 entries, 0 to 374297
Data columns (total 18 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   bookid            9013 non-null    float64
 1   title             374298 non-null  object 
 2   authors           374296 non-null  object 
 3   avgreviews        9013 non-null    float64
 4   isbn              279116 non-null  object 
 5   isbn13            278993 non-null  object 
 6   languagecode      9013 non-null    object 
 7   numpages          9013 non-null    float64
 8   nreviews          9013 non-null    float64
 9   textreviewscount  9013 non-null    float64
 10  publisher         374288 non-null  object 
 11  publicationyear   374296 non-null  float64
 12  image-url-s       272549 non-null  object 
 13  image-url-m       272549 non-null  object 
 14  image-url-l       272549 non-null  object 
 15  description       77417 non-null   object 
 16  category          83

In [2]:
import pyodbc

# Thông tin kết nối
server = 'DESKTOP-G05LG93\\SQLEXPRESS'
database = 'DataWarehouse'
username = ''
password = ''

# Chuỗi kết nối
connection_string = f'DRIVER={{SQL Server}};SERVER={server};DATABASE={database};UID={username};PWD={password}'

# Kết nối tới SQL Server
try:
    conn = pyodbc.connect(connection_string)
    print("Kết nối thành công!")
    query = "select * from dimBooks"
    df = pd.read_sql(query, conn)
    print(df.head())
    
except pyodbc.Error as e:
    print(f": {e}")


# Đóng kết nối
conn.close()


Kết nối thành công!
Empty DataFrame
Columns: [BookID, Title, Authors, ISBN, ISBN13, LanguageCode, NumPages, PublicationYear, Publisher, Description, Category]
Index: []


C:\Users\buida\AppData\Local\Temp\ipykernel_13780\941499857.py:17: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


In [6]:
import pandas as pd
import numpy as np
import pyodbc

# Thông tin kết nối
server = 'DESKTOP-G05LG93\\SQLEXPRESS'
database = 'DataWarehouse2'
username = ''
password = ''

# Chuỗi kết nối
connection_string = f'DRIVER={{SQL Server}};SERVER={server};DATABASE={database};UID={username};PWD={password}'

# Kết nối tới SQL Server
conn = pyodbc.connect(connection_string)
print("Kết nối thành công!")

# Đọc dữ liệu từ tệp CSV và thay thế NaN bằng None
df = pd.read_csv("data/merged_books2.csv", delimiter=",")
df = df.where(pd.notnull(df), -1)

# Chuyển đổi kiểu dữ liệu của các cột
df['publicationyear'] = df['publicationyear'].astype('Int64')
df['numpages'] = df['numpages'].astype('Int64')
df['nreviews'] = df['nreviews'].astype('Int64')
df['textreviewscount'] = df['textreviewscount'].astype('Int64')
df['price'] = df['price'].astype('Float64')
df['avgreviews'] = df['avgreviews'].astype('Float64')

# Tạo một con trỏ (cursor)
cursor = conn.cursor()

# Lặp qua từng dòng trong DataFrame và thêm vào cơ sở dữ liệu
for index, row in df.iterrows():
    # Thêm dữ liệu vào bảng dimBooks
    insert_query = """
    INSERT INTO dimBooks (Title, Authors, ISBN, ISBN13, LanguageCode, NumPages, PublicationYear, Publisher, Description, Category)
    VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
    """
    values = (
        row['title'],
        row['authors'],
        row['isbn'],
        row['isbn13'],
        row['languagecode'],
        row['numpages'],
        row['publicationyear'],
        row['publisher'],
        row['description'],
        row['category']
    )
    cursor.execute(insert_query, values)

    # Thêm dữ liệu vào bảng factBookDetails
    insert_query = """
    INSERT INTO factBookDetails (BookID, Price, AvgReviews, NReviews, Image_url_s, Image_url_m, Image_url_l)
    VALUES (?, ?, ?, ?, ?, ?, ?)
    """
    values = (
        # Lấy giá trị BookID từ bảng dimBooks
        cursor.execute("SELECT SCOPE_IDENTITY()").fetchone()[0],
        row['price'],
        row['avgreviews'],
        row['nreviews'],
        row['image-url-s'],
        row['image-url-m'],
        row['image-url-l']
    )
    cursor.execute(insert_query, values)

# Lưu các thay đổi
conn.commit()
print("Dữ liệu đã được ghi vào cơ sở dữ liệu thành công!")

# Đóng kết nối
conn.close()


Kết nối thành công!


C:\Users\buida\AppData\Local\Temp\ipykernel_13780\3897731325.py:19: DtypeWarning: Columns (4,5,6,12,13,14,15,16) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("data/merged_books2.csv", delimiter=",")


Dữ liệu đã được ghi vào cơ sở dữ liệu thành công!


In [7]:
import pandas as pd
import numpy as np
import pyodbc

# Thông tin kết nối
server = 'DESKTOP-G05LG93\\SQLEXPRESS'
database = 'DataWarehouse3'
username = ''
password = ''

# Chuỗi kết nối
connection_string = f'DRIVER={{SQL Server}};SERVER={server};DATABASE={database};UID={username};PWD={password}'

# Kết nối tới SQL Server
conn = pyodbc.connect(connection_string)
print("Kết nối thành công!")

# Đọc dữ liệu từ tệp CSV và thay thế NaN bằng None
df = pd.read_csv("data/merged_books3.csv", delimiter=",")
df = df.where(pd.notnull(df), -1)

# Chuyển đổi kiểu dữ liệu của các cột
df['publicationyear'] = df['publicationyear'].astype('Int64')
df['numpages'] = df['numpages'].astype('Int64')
df['nreviews'] = df['nreviews'].astype('Int64')
df['textreviewscount'] = df['textreviewscount'].astype('Int64')
df['price'] = df['price'].astype('Float64')
df['avgreviews'] = df['avgreviews'].astype('Float64')

# Tạo một con trỏ (cursor)
cursor = conn.cursor()

# Lặp qua từng dòng trong DataFrame và thêm vào cơ sở dữ liệu
for index, row in df.iterrows():
    # Thêm dữ liệu vào bảng dimBooks
    # Thêm dữ liệu vào bảng dimBooks
    insert_query_dimBooks = """
    INSERT INTO dimBooks (Title, Authors, ISBN, ISBN13, LanguageCode, NumPages, PublicationYear, Publisher, Description, Category)
    VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
    """
    values_dimBooks = (
        row['title'],
        row['authors'],
        row['isbn'],
        row['isbn13'],
        row['languagecode'],
        row['numpages'],
        row['publicationyear'],
        row['publisher'],
        row['description'],
        row['category']
    )
    cursor.execute(insert_query_dimBooks, values_dimBooks)

    # Lấy giá trị BookID mà vừa được thêm vào bảng dimBooks
    cursor.execute("SELECT IDENT_CURRENT('dimBooks')")
    BookID = cursor.fetchone()[0]

    # Thêm dữ liệu vào bảng factBookDetails
    insert_query_factBookDetails = """
    INSERT INTO factBookDetails (BookID, Price, AvgReviews, NReviews, Image_url_s, Image_url_m, Image_url_l)
    VALUES (?, ?, ?, ?, ?, ?, ?)
    """
    values_factBookDetails = (
        BookID,
        row['price'],
        row['avgreviews'],
        row['nreviews'],
        row['image-url-s'],
        row['image-url-m'],
        row['image-url-l']
    )
    cursor.execute(insert_query_factBookDetails, values_factBookDetails)

# Lưu các thay đổi
conn.commit()
print("Dữ liệu đã được ghi vào cơ sở dữ liệu thành công!")

# Đóng kết nối
conn.close()


Kết nối thành công!


C:\Users\buida\AppData\Local\Temp\ipykernel_13780\3180168703.py:19: DtypeWarning: Columns (4,5,6,10,13,15,16,17,18,19) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("data/merged_books3.csv", delimiter=",")


Dữ liệu đã được ghi vào cơ sở dữ liệu thành công!
